In [19]:

import cv2
import time
from google.colab.patches import cv2_imshow
from google.colab import drive
from ultralytics import YOLO

# Mount Google Drive
drive.mount('/content/drive')

model_path = '/content/drive/MyDrive/HITAI/kabbadi/models/best_yolov8_kabbadi_mainplayer.pt'
video_path = '/content/drive/MyDrive/HITAI/kabbadi/datasetv2/video_test.mp4'

# Initialize model
model = YOLO(model_path)

# Open video capture
cap = cv2.VideoCapture(video_path)
if not cap.isOpened():
    print("Error opening video file")
    exit()

# Video writer setup
fps = cap.get(cv2.CAP_PROP_FPS)
width = int(cap.get(cv2.CAP_PROP_FRAME_WIDTH))
height = int(cap.get(cv2.CAP_PROP_FRAME_HEIGHT))
fourcc = cv2.VideoWriter_fourcc(*'mp4v')
out = cv2.VideoWriter('output_video.mp4', fourcc, fps, (width, height))

# Initialize variables
player1_start_time = 0.0
player2_start_time = 0.0
frame_center_x = width // 2
player1_consecutive_frames = 0
player2_consecutive_frames = 0

while cap.isOpened():
    success, frame = cap.read()
    if not success:
        break

    results = model(frame)
    annotated_frame = results[0].plot(labels=False)

    current_time = time.time()
    detected_player = None

    for r in results:
        boxes = r.boxes
        for box in boxes:
            class_id = int(box.cls)
            if class_id not in [0, 1]:
                continue

            x1, y1, x2, y2 = map(int, box.xyxy[0].tolist())
            center_x = int((x1 + x2) / 2)
            player_label = 'team_1' if class_id == 0 else 'team_2'
            detected_player = player_label

            duration = 0.0
            if player_label == 'team_1':
                if center_x >= frame_center_x:
                    player1_consecutive_frames +=1
                    player2_consecutive_frames = 0
                    if player1_consecutive_frames >=5 and player1_start_time == 0.0:
                      player1_start_time = current_time
                      player2_start_time = 0.0
                    if player1_start_time != 0.0:
                        duration = round((current_time - player1_start_time), 2)
                else:
                    player1_consecutive_frames = 0

            elif player_label == 'team_2':
                if center_x <= frame_center_x:
                    player2_consecutive_frames +=1
                    player1_consecutive_frames = 0
                    if player2_consecutive_frames >=5 and player2_start_time == 0.0:
                      player2_start_time = current_time
                      player1_start_time = 0.0
                    if player2_start_time != 0.0:
                        duration = round((current_time - player2_start_time), 2)
                else:
                    player2_consecutive_frames = 0

            text_color = (0, 255, 0) if duration < 20 else ((0, 0, 255) if duration > 30 else (0, 255, 255))
            cv2.putText(annotated_frame, f'Time: {duration}s', (x1, y1 - 10), cv2.FONT_HERSHEY_SIMPLEX, 2, (0,0,0), 6)
            cv2.putText(annotated_frame, f'Time: {duration}s', (x1, y1 - 10), cv2.FONT_HERSHEY_SIMPLEX, 2, text_color, 3)
            break #oly display the closest player
    out.write(annotated_frame)

cap.release()
out.release()

Streaming output truncated to the last 5000 lines.
0: 736x1280 1 center, 12.7ms
Speed: 7.4ms preprocess, 12.7ms inference, 1.4ms postprocess per image at shape (1, 3, 736, 1280)

0: 736x1280 1 center, 12.7ms
Speed: 6.9ms preprocess, 12.7ms inference, 1.5ms postprocess per image at shape (1, 3, 736, 1280)

0: 736x1280 1 center, 12.7ms
Speed: 6.8ms preprocess, 12.7ms inference, 1.4ms postprocess per image at shape (1, 3, 736, 1280)

0: 736x1280 1 center, 12.7ms
Speed: 6.6ms preprocess, 12.7ms inference, 1.4ms postprocess per image at shape (1, 3, 736, 1280)

0: 736x1280 1 center, 12.9ms
Speed: 10.7ms preprocess, 12.9ms inference, 1.4ms postprocess per image at shape (1, 3, 736, 1280)

0: 736x1280 1 center, 12.7ms
Speed: 6.8ms preprocess, 12.7ms inference, 1.4ms postprocess per image at shape (1, 3, 736, 1280)

0: 736x1280 1 center, 12.8ms
Speed: 6.7ms preprocess, 12.8ms inference, 1.5ms postprocess per image at shape (1, 3, 736, 1280)

0: 736x1280 1 center, 12.7ms
Speed: 7.2ms preprocess